Table as_16_q: Asylum seekers in receipt of Section 95 support, by local authority, as at end of quarter

In [57]:
from gssutils import *

scraper = Scraper('https://www.gov.uk/government/statistics/immigration-statistics-october-to-december-2017-data-tables')
scraper

## Immigration statistics, October to December 2017: data tables second edition

Listing of the data tables included in 'Immigration statistics, October to December 2017’.

### Description

Listing of the data tables included in [‘Immigration Statistics, October to
December 2017’](https://www.gov.uk/government/statistics/immigration-
statistics-october-to-december-2017).

Data is in yearly or quarterly divisions.

To view and use ‘ODS’ files, OS X users can [download
OpenOffice](http://www.openoffice.org/download/).



### Distributions

1. Sponsorship data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683358/sponsorship-oct-dec-2017-tables.ods))
1. Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 1 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683359/entry-visas1-oct-dec-2017-tables.ods))
1. Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 2 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681238/entry-visas2-oct-dec-2017-tables.ods))
1. Entry clearance visas granted outside the UK data tables immigration statistics October to December 2017 volume 3 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681241/entry-visas3-oct-dec-2017-tables.ods))
1. Admissions data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681242/admissions-oct-dec-2017-tables.ods))
1. Extensions of stay data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681243/extensions-oct-dec-2017-tables.ods))
1. Settlement data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681244/settlement-oct-dec-2017-tables.ods))
1. Citizenship data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683360/citizenship-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 1 second edition ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691963/asylum1-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 2 second edition ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691964/asylum2-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 3 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691965/asylum3-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 4 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691967/asylum4-oct-dec-2017-tables.ods))
1. Asylum data tables immigration statistics October to December 2017 volume 5 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/691969/asylum5-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 1 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681246/returns1-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 2 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683361/returns2-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 3 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/683362/returns3-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 4 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681250/returns4-oct-dec-2017-tables.ods))
1. Returns data tables immigration statistics October to December 2017 volume 5 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681251/returns5-oct-dec-2017-tables.ods))
1. Detention data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681703/detention-oct-dec-2017-tables.ods))
1. European Economic Area data tables immigration statistics October to December 2017 ([ODF Spreadsheet](https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/681253/eea-oct-dec-2017-tables.ods))


In [58]:
dist = scraper.distribution(
    title='Asylum data tables immigration statistics October to December 2017 volume 4'
)
sheets = dist.as_pandas()
sheets.keys()

dict_keys(['Cover_sheet', 'Contents', 'Notes', 'as_14', 'as_14_q', 'as_15', 'as_16_q', 'as_17_q', 'as_18_q', 'as_19_q', 'as_20_q'])

Metadata is in the spreadsheet 'Contents' tab.

In [59]:
TAB_NAME = 'as_16_q'

contents = sheets['Contents'].iloc[7:].copy()
contents.rename(columns=contents.iloc[0], inplace=True)
contents.drop(contents.index[0], inplace=True)
metadata = contents[contents['Table'] == TAB_NAME].iloc[0]
display(metadata)
scraper.dataset.comment = f'{scraper.dataset.title}: table "{TAB_NAME}"'
scraper.dataset.title = metadata['Title']
scraper.dataset.issued = metadata['Last updated']
scraper.dataset.updateDueOn = metadata['Next planned update']
del scraper.dataset.description
scraper.dataset

Table                                                             as_16_q
Title                   Asylum seekers in receipt of Section 95 suppor...
Period covered                                         Q4 2003 to Q4 2017
National Statistics?                                                  Yes
Last updated                                          2018-02-22 00:00:00
Next planned update                                   2018-05-24 00:00:00
Notes                                                                    
Name: 35, dtype: object

In [60]:
observations = sheets[TAB_NAME].iloc[3:, :7].copy()
observations.rename(columns= observations.iloc[0], inplace=True)
observations.drop(observations.index[0], inplace = True)
observations.drop('Region', axis = 1, inplace = True)
Final_table = pd.melt(observations,
                       ['Quarter','Local Authority'],
                       var_name="Total supported under Section 95",
                       value_name="Value")
Final_table.Value.dropna(inplace =True)
Final_table['Unit'] = 'asylum-seekers'
Final_table['Measure Type'] = 'Count'

In [61]:
Final_table['Value'] = Final_table['Value'].map(lambda x : ''
                                                   if (x == '.') | (x == 'z') | ( x == ':')
                                                    else x )

In [62]:
import numpy as np
Final_table['Value'].replace('', np.nan, inplace=True)
Final_table.dropna(subset=['Value'], inplace=True)
Final_table['Value'] = Final_table['Value'].apply(lambda x: pd.to_numeric(x, downcast='integer'))
Final_table['Value'] = Final_table['Value'].astype(int)

In [63]:
Final_table.rename(columns={'Quarter': 'Period'}, inplace=True)
Final_table.rename(columns={'Total supported under Section 95': 'Section 95 Support'}, inplace=True)
Final_table['Period'] = Final_table['Period'].astype(str)
Final_table = Final_table[Final_table['Period'] != '']
Final_table['Period'] = 'quarter/' + Final_table['Period'].map(lambda cell: cell.replace(' ','-'))
Final_table['Section 95 Support'] = Final_table['Section 95 Support'].map(lambda cell: cell.replace('In receipt of subsistence \nonly', 'In receipt of subsistence only'))

In [64]:
Final_table['Local Authority'] = Final_table['Local Authority'].str.lstrip('*')

In [65]:
Final_table['Local Authority'] = Final_table['Local Authority'].map(
    lambda x: {
        'Total East Midlands':'East Midlands',
        'Total East of England':'East of England',
        'Total London':'London',
        'Total North East':'North East',
        'Total North West':'North West',
        'Total Northern Ireland':'Northern Ireland',
        'Total Scotland':'Scotland',
        'Total South East':'South East',
        'Total South West':'South West',
        'Total Wales':'Wales',
        'Total West Midlands':'West Midlands',
        'Total Yorkshire and The Humber':'Yorkshire and The Humber',
        'Total England' : 'England',
        'Total Other and Unknown' : 'Other and Unknown'
        }.get(x, x))

In [66]:
Final_table['Section 95 Support'] = Final_table['Section 95 Support'].map(
    lambda x: {
        'Total supported under Section 95':'total-supported',
        'In receipt of subsistence only':'subsistence-only',
        'In dispersed accommodation':'dispersed-accommodation',
        'Disbenefited':'disbenefited',
        }.get(x, x))

In [67]:
# Take the Labels:Codes pairs from ./gss-codes.csv and create a dictionary
# use that to convert 'Local Authority' labels to codes
lookup_dict = {}
name_to_label_dataframe = pd.read_csv("./gss-codes.csv")
for _, row in name_to_label_dataframe.iterrows():
    lookup_dict.update({row["Label"]: row["Code"]})
Final_table["Local Authority"] = Final_table["Local Authority"].map(lambda x: lookup_dict[x])
Final_table.rename(columns={'Local Authority':'Geography'}, inplace=True)

In [68]:
Final_table = Final_table[['Period','Geography','Section 95 Support','Measure Type','Value','Unit']]

In [69]:
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

Final_table.drop_duplicates().to_csv(destinationFolder / f'{TAB_NAME}.csv', index = False)

In [70]:
from gssutils.metadata import THEME

scraper.dataset.family = 'migration'
scraper.dataset.theme = THEME['population']
scraper.set_base_uri('http://gss-data.org.uk')
scraper.set_dataset_id(f'migration/ho-asylum/{TAB_NAME}')
with open(destinationFolder / f'{TAB_NAME}.csv-metadata.trig', 'wb') as metadata:
    metadata.write(scraper.generate_trig())

In [71]:
schema = CSVWMetadata('https://ons-opendata.github.io/ref_migration/')
schema.create(destinationFolder / f'{TAB_NAME}.csv', destinationFolder / f'{TAB_NAME}.csv-schema.json')

In [72]:
Final_table

,Period,Geography,Section 95 Support,Measure Type,Value,Unit
0,quarter/2003-Q4,K02000001,total-supported,Count,80123,asylum-seekers
1,quarter/2003-Q4,E12000004,total-supported,Count,4524,asylum-seekers
2,quarter/2003-Q4,E12000006,total-supported,Count,1944,asylum-seekers
3,quarter/2003-Q4,E12000007,total-supported,Count,24916,asylum-seekers
4,quarter/2003-Q4,E12000001,total-supported,Count,5172,asylum-seekers
5,quarter/2003-Q4,E12000002,total-supported,Count,9093,asylum-seekers
6,quarter/2003-Q4,N92000002,total-supported,Count,153,asylum-seekers
8,quarter/2003-Q4,S92000003,total-supported,Count,5818,asylum-seekers
9,quarter/2003-Q4,E12000008,total-supported,Count,3208,asylum-seekers
10,quarter/2003-Q4,E12000009,total-supported,Count,1668,asylum-seekers
